<a href="https://colab.research.google.com/github/rizveeredwan/Annoying-Mute-Line/blob/master/mcts_hamiltonian_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Implemention of generic MCTS with Hamiltonian function 

In [ ]:
import os, sys
system = "colab"
if system == "local":
  BASE_PATH = os.path.join('.') # 
elif system == "colab":
  BASE_PATH = os.path.join('/content', "gdrive", 'MyDrive', 'Research', 'MADRL')
  #to be able to interact with Google Drive's operating system
  from google.colab import drive 
  #drive is a module that allows us use Python to interact with google drive
  drive.mount('/content/gdrive') 
  #mounting google drive allows us to work with its contents
  nb_path = '/content/notebooks'
  if os.path.exists('/content/gdrive/My Drive/Colab Notebooks') is False:
    os.symlink('/content/gdrive/My Drive/Colab Notebooks', nb_path)
    sys.path.insert(0, nb_path)  # or append(nb_path)
  sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

Mounted at /content/gdrive


In [ ]:
!pip install torch_geometric
!pip install torch_sparse
!pip install torch_scatter
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=4f6a7d6cb453de8f2a305e332497a1c3102afacf40efd00f4327d57611143298
  Stored in directory: /root/.cache/pip/wheels/cd/7d/6b/17150450b80b4a3656a84330e22709ccd8dc0f8f4773ba4133
Successfully built torch_geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_sparse: filename=torch_sparse-0.6.17-cp39-cp39-linux_x86_64.whl size=1082944 sha256=658ec7bc81b04b432e9029995ae2707927389ca3ded

In [ ]:
import random 
import numpy as np 
from math import sqrt , log
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch.nn import Linear, Parameter, Module, Embedding, Softmax
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import torch.nn.functional as F

import networkx as nx
import dgl
import random 
import math

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
def graph_read(file_name=os.path.join('/content', "gdrive", 'MyDrive', 'Research', 'MADRL', 'Graph Dataset', "500_799.txt")):
  # reading the text file 
  if os.path.exists(file_name):
    edge_index = [[],[]]
    with open(file_name, 'r', encoding='utf-8') as f:
      lines = f.readline().strip().split(' ')
      n, e = int(lines[0]), int(lines[1])
      adj_list = []
      for i in range(0, n):
        adj_list.append([])
      for i in range(0, e):
        lines = f.readline().strip().split(' ')
        a, b = int(lines[0]), int(lines[1]) 
        adj_list[a-1].append(b-1)
        adj_list[b-1].append(a-1)
        edge_index[0].append(a-1)
        edge_index[1].append(b-1)
        edge_index[0].append(b-1)
        edge_index[1].append(a-1)
      edge_index = torch.tensor(edge_index)
      return adj_list,edge_index 
def construct_q_matrix(adj_list):
  Q = torch.zeros((len(adj_list), len(adj_list)))
  for i in range(0, len(adj_list)):
    Q[i][i] =  len(adj_list[i])
    for j in range(0, len(adj_list[i])):
      if adj_list[i][j] > i:
        Q[i][adj_list[i][j]] = -2 
  return Q 

In [ ]:
class State:
  def __init__(self):
    self.n = 0 # count 
    self.w = 0 # total reward achieved from bottom 
    self.labels = {}
    self.child = {} # child node 
    self.parent = None 
    self.edge_prob = 1.0 

In [ ]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, norm=norm)

        # Step 6: Apply a final bias vector.
        out += self.bias

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

In [ ]:
class NNet(Module):
  def __init__(self, n, in1, dropout_frac=0.5):
    super().__init__()
    self.dropout_frac = dropout_frac
    self.n_nodes = n

    self.embed_layer = Embedding(n, in1) # embedding layer

    self.conv1 = GCNConv(in_channels=in1, out_channels=in1)
    self.conv2 = GCNConv(in_channels=in1, out_channels=1)

    self.linear = Linear(1, 1) # theta1 * xv 
    self.linear2 = Linear(1, 1) # theta2 * mv 
    self.linear3 = Linear(self.n_nodes, 2 * self.n_nodes) # theta3 (theta1 * xv + theta2 * mv) 

  def forward(self, node_list_tensor, edge_index, X):
    inputs = self.embed_layer(node_list_tensor)
    mv = self.conv1(inputs, edge_index) # Embeddings, edge_index 
    mv = torch.relu(mv)
    mv = F.dropout(mv, p=self.dropout_frac)
    mv = self.conv2(mv, edge_index) # Found the mv here

    values = self.linear(X)
    values2 = self.linear2(mv) 
    values3 = torch.relu(values+values2)
    #print("age ", values3. shape, values3)
    values3 = torch.squeeze(values3)
    #print("pore ", values3.shape, values3)
    values4 = self.linear3(values3)
    return torch.sigmoid(values4)

  def calculate_loss(self, prob_distribution, mcts_node, adj_list):
    probs = prob_distribution[self.n_nodes:]
    #print("probs ", probs)
    loss = 0
    for i in range(0, len(adj_list)):
      if mcts_node.labels.get(i) is not None: # already calculate probability 
        loss = loss + mcts_node.labels[i] * len(adj_list[i]) * mcts_node.labels[i]
        prob_i = mcts_node.labels[i]
      else:
        loss = loss + probs[i] * len(adj_list[i]) *probs[i]
        prob_i = probs[i]
      for j in range(0, len(adj_list[i])):
        v = adj_list[i][j]
        if v > i:
          if mcts_node.labels.get(v) is not None: # custom probability 
            loss = loss + prob_i * (-2) * mcts_node.labels[v]
          else:
            #print("v ", v)
            #print("probs v", probs[v])
            loss = loss + prob_i * (-2) * probs[v] 
    return loss 

  def calculate_loss_using_vector_op(self, prob_distribution, mcts_node, Q):
    probs = prob_distribution[self.n_nodes:]
    #print("probs ", probs)
    bool_tensor = torch.ones(Q.shape[0])
    for i in range(0, probs.shape[0]):
      bool_tensor[i] = probs[i] 
      #print(bool_tensor[i].requires_grad)
    for key in mcts_node.labels:
      bool_tensor[key] = 0 * probs[key] + mcts_node.labels[key]
    # print(probs, probs.shape)
    value = torch.matmul(bool_tensor, Q) 
    loss = torch.matmul(value, bool_tensor)
    return loss * -1 # to minimize 


In [ ]:
def function_plot(data, title='loss variation', ylabel='loss', xlabel='epoch', loc="upper left", fig_name=None):
  plt.plot(data)
  plt.title(title)
  plt.ylabel(ylabel)
  plt.xlabel(xlabel)
  plt.show()
  if fig_name is not None:
    plt.savefig(fig_name)

In [ ]:
NUM_NODES = 1
def dict_copy(par_state, child_state):
  for key in par_state.labels:
    child_state.labels[key] = par_state.labels[key]
  return 

def zero_tensor(n):
  xv = []
  for i in range(0, n):
    xv.append([0.0])
  return torch.tensor(xv)

def make_node_list_tensor(n):
  node_list_tensor = []
  for i in range(n):
    node_list_tensor.append(i)
  return torch.tensor(node_list_tensor) 

def score_calculation(X, Q):
  val = torch.matmul(X, Q)
  val2 = torch.matmul(val, X)
  return val2 

def selection(current_state, adj_list, C):
  global NUM_NODES
  if len(current_state.child) == 0: # leaf node 
    if current_state.n == 0: # first time visit 
      return current_state
    else:
      return current_state # multiple times visit but last node 
  else: # non leaf node, choose based on the value of UCB1
    best_ucb, node = None, None 
    for ch in current_state.child:
      child = current_state.child[ch]
      if child.n == 0:
        return selection(current_state=child, adj_list=adj_list, C=C)
      else:
        vi = child.w/(1.0 * child.n)
        ucb1 = vi + C * child.edge_prob * sqrt(1.0 * log(current_state.n)/child.n)
        if best_ucb is None:
          best_ucb = ucb1 
          node = child  
        elif best_ucb < ucb1:
          best_ucb = ucb1 
          node = child 
    return selection(current_state=node, adj_list=adj_list, C=C)

def apply_thresholding(prob_distribution, Q, threshold=0.5):
  values = torch.zeros(Q.shape[0])
  for i in range(0, prob_distribution.shape[0]):
    if prob_distribution[i] >= threshold:
      values[i] = 1
  v = torch.matmul(values, Q)
  reward = torch.matmul(v, values)
  return reward, values  

def train(current_state, nnet=None, epoch=1000, node_list_tensor=[], edge_index=None, X=None, Q=None, patience=30, lr=0.001, input_feature=7):
  if nnet is None: # if none, new creation, otherwise just progress onward 
    nnet = NNet(n=Q.shape[0], in1=input_feature)
  optimizer = torch.optim.Adam(nnet.parameters(), lr=lr)
  X = zero_tensor(Q.shape[0])
  for key in current_state.labels:
    X[key] = 1.0
  min_loss = None 
  best_output = None 
  count = 0
  for i in range(0, epoch):
    prob_distribution = nnet(node_list_tensor=node_list_tensor, edge_index=edge_index, X=X)
    loss = nnet.calculate_loss_using_vector_op(prob_distribution=prob_distribution, mcts_node=current_state, Q=Q)
    if i%1000 == 0:
      print(f"iteration {i} loss = {loss} prob {prob_distribution[Q.shape[0]: Q.shape[0]+30]}")

    optimizer.zero_grad()
    loss.backward()
    optimizer.step() 
    
    if min_loss is None:
      min_loss = loss 
      best_output = prob_distribution 
      count = 0
    elif min_loss > loss:
      min_loss = loss 
      best_output = prob_distribution 
      count = 0 
    elif min_loss <= loss:
      count += 1 
      if count >= patience:
        return best_output 
  return best_output 

def make_X_tensor(n, current_state):
  X = zero_tensor(n=n)
  for key in current_state.child:
    X[key] = 1.0 
  return X 

def rollout(current_state, nnet, edge_index, Q, node_list_tensor, epoch=100, patience=30, lr=0.001, input_feature=7, threshold=0.5):
  # Apply 
  X = make_X_tensor(n=Q.shape[0], current_state=current_state)
  best_output = train(current_state=current_state, nnet=nnet, epoch=epoch, node_list_tensor=node_list_tensor, edge_index=edge_index, X=X, Q=Q, patience=patience, lr=lr, input_feature=input_feature)
  transition = best_output[0:Q.shape[0]]
  probs = best_output[Q.shape[0]:]
  score, labels = apply_thresholding(prob_distribution=probs, Q=Q, threshold=threshold)
  return transition, score, labels 

def backpropagation(leaf_node, w):
  leaf_node.w += w 
  leaf_node.n += 1
  if leaf_node.parent is not None:
    backpropagation(leaf_node.parent, w)
  return 

def policy_return(root, nnet=None, epoch=1000,  node_list_tensor=None, edge_index=None, Q=None, patience=30, lr=0.001, input_feature=7, threshold=0.5):
  current = root 
  labels = []
  assert(Q is not None)
  X = zero_tensor(Q.shape[0])
  while True:
    if len(current.child) == 0:
      if len(current.labels) < Q.shape[0]: # some are not calculated, need neural net 
        X = make_X_tensor(n=Q.shape[0], current_state=current)
        best_output  = train(current_state=current, nnet=nnet, epoch=epoch, node_list_tensor=node_list_tensor, edge_index=edge_index, X=X, Q=Q, patience=patience, lr=lr, input_feature=input_feature)
        reward, values = apply_thresholding(prob_distribution=best_output[Q.shape[0]:], Q=Q, threshold=threshold)
        return reward, values 
      else: # all are calculated 
         labels = torch.zeros(len(adj_list))
         for i in range(0, len(adj_list)):
           labels[i] = current.labels[i] 
         reward, values = apply_thresholding(prob_distribution=labels, Q=Q, threshold=threshold)
      return reward, values
    else: # has child 
      best_node, best_v = None, None  
      for ch in current.child:
        if current.child[ch].n > 0:
          v = current.child[ch].w / (1.0 * current.child[ch].n)
          if best_v is None:
            best_v = v 
            best_node = current.child[ch]
          elif best_v < v:
            best_v = v 
            best_node = current.child[ch]
        else:
          best_node = current.child[ch]
          break 
      current = best_node 

def violation_constraint(adj_list, labels):
  violation = 0
  for i in range(0, len(adj_list)):
    v = adj_list[i][0]
    if v > i and labels[i] == labels[v]:
      violation += 1
  return violation 

def create_child_nodes(selected_node, transition, Q):
  global NUM_NODES
  for i in range(Q.shape[0]):
        if selected_node.labels.get(i) is None:
          new_state = State()
          new_state.labels[i] = 0 # label: 0 
          selected_node.child[(i, 0)] = new_state 
          new_state.parent_node = selected_node
          new_state.edge_prob = 1.0 # transition[i] 
          dict_copy(par_state=selected_node, child_state = new_state)
          NUM_NODES += 1

          new_state = State()
          new_state.labels[i] = 1 # label: 0 
          selected_node.child[(i, 1)] = new_state 
          new_state.parent_node = selected_node
          new_state.edge_prob = 1.0 # transition[i] 
          dict_copy(par_state=selected_node, child_state = new_state)
          NUM_NODES += 1


def mcts_algo(adj_list, C, nnet=None, iteration=30000, Q=None, epoch=10000, patience=30, lr=0.001, input_feature=7, threshold=0.5):
  global NUM_NODES
  root = State() # root node 
  rewards = []
  violations = []
  node_list_tensor = make_node_list_tensor(Q.shape[0])

  for i in range(0, iteration):
    # node selection 
    selected_node = selection(current_state=root, adj_list=adj_list, C=C)
    # rollout (heavy)
    transition, score, labels = rollout(current_state=selected_node, nnet=nnet, edge_index=edge_index, Q=Q, node_list_tensor=node_list_tensor, epoch=epoch, patience=patience, lr=lr, input_feature=input_feature, 
            threshold=threshold)
    # backpropagation the values 
    backpropagation(leaf_node=selected_node, w=score) 
    # creating child nodes 
    create_child_nodes(selected_node=selected_node, transition=transition, Q=Q)
    print("rollout_val ", score)
    
    reward, labels = policy_return(root=root, nnet=nnet, epoch=epoch, node_list_tensor=node_list_tensor, edge_index=edge_index, Q=Q, patience=patience, lr=lr, input_feature=input_feature, threshold=threshold)
    rewards.append(score_calculation(labels, Q))
    violations.append(violation_constraint(adj_list, labels))
  
  print(labels)
  print(reward, violation_constraint(adj_list, labels))
  function_plot(data=rewards, title='reward variation', ylabel='reward', xlabel='iteration', loc="upper left", fig_name=None)
  function_plot(data=violations, title='violation variation', ylabel='violation', xlabel='iteration', loc="upper left", fig_name=None)


In [ ]:
data_file = "500_799"
adj_list, edge_index = graph_read(file_name=os.path.join(BASE_PATH, 'Graph Dataset', data_file+'.txt'))
Q = construct_q_matrix(adj_list=adj_list)
print(Q)
print(adj_list)
print(edge_index)
single_nnet = NNet(n=Q.shape[0], in1=7)
mcts_algo(adj_list=adj_list, C=2, nnet=single_nnet, iteration=10000, Q=Q, epoch=10000, patience=100, lr=0.001, input_feature=7, threshold=0.5)
print(NUM_NODES)

Streaming output truncated to the last 5000 lines.
       grad_fn=<SliceBackward0>)
rollout_val  tensor(561.)
iteration 0 loss = -559.0 prob tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 2.8241e-12,
        1.0000e+00, 2.1193e-12, 4.2310e-12, 1.0000e+00, 4.2324e-12, 1.0000e+00,
        1.2129e-12, 4.2351e-12, 1.0000e+00, 1.0000e+00, 8.4421e-12, 1.0000e+00,
        2.8229e-12, 1.0000e+00, 2.8255e-12, 2.8238e-12, 1.0000e+00, 1.0000e+00,
        2.8235e-12, 4.2306e-12, 4.2344e-12, 1.0000e+00, 1.0000e+00, 1.0000e+00],
       grad_fn=<SliceBackward0>)
iteration 0 loss = -559.0 prob tensor([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 2.8236e-12,
        1.0000e+00, 2.1189e-12, 4.2302e-12, 1.0000e+00, 4.2315e-12, 1.0000e+00,
        1.2126e-12, 4.2343e-12, 1.0000e+00, 1.0000e+00, 8.4404e-12, 1.0000e+00,
        2.8223e-12, 1.0000e+00, 2.8250e-12, 2.8233e-12, 1.0000e+00, 1.0000e+00,
        2.8230e-12, 4.2298e-12, 4.2336e-12, 1.0000e+00, 1.0000e+00, 1.0000

In [ ]:
X = zero_tensor(n=len(adj_list))
node_list_tensor = make_node_list_tensor(len(adj_list))
mcts_node = State()
mcts_node.labels[2] = 1
mcts_node.labels[1] = 1
train(current_state=mcts_node, epoch=100, node_list_tensor=node_list_tensor, edge_index=edge_index, X=X, Q=Q, patience=30, lr=0.001, input_feature=7)